In [4]:
import requests
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
from lxml import etree

In [5]:
jugador = "Lamine Yamal"
fecha_inicio = "2022-01-01"
fecha_fin = "2023-01-31"

url = f"https://news.google.com/search?q={jugador.replace(' ', '%20')}+after:{fecha_inicio}+before:{fecha_fin}"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

elements = soup.find_all('c-wiz', {'jsrenderer': 'ARwRbe'})

In [6]:
l_href = []

for element in elements:
    links = element.find_all('a')

    for link in links:
        href = link.get('href')

        if href:
            l_href.append(href)

print(l_href)


['https://support.google.com/googlenews?p=consumer_info&hl=en', './read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmpMaDRGU25MQkVzbDA3eXhfQ1AydGR1cFk4RVlxazlzR1JvaXBoVmpUT0tNZXNnX2ZPUEJJS3BmdmxJTGhfdS1KS0UyNzFVRnZhY250eFo1dkROZ0NWb0RzN25OY1JyM0k0OGRwVkJuS1VrRkhSc1FnMGVFQzFwNGtLMA?hl=en-US&gl=US&ceid=US%3Aen', './read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmp

In [7]:
def get_google_params(url):
    response = requests.get(url, headers={})
    print(response.status_code)
    tree = etree.HTML(response.text)

    sign = tree.xpath('//c-wiz/div/@data-n-a-sg')[0]
    ts = tree.xpath('//c-wiz/div/@data-n-a-ts')[0]
    source = tree.xpath('//c-wiz/div/@data-n-a-id')[0]
    return source, sign, ts




def get_origin_url(source, sign, ts):
    url = f"https://news.google.com/_/DotsSplashUi/data/batchexecute"
    req_data = [[[
        "Fbv4je",
        f"[\"garturlreq\",[[\"zh-HK\",\"HK\",[\"FINANCE_TOP_INDICES\",\"WEB_TEST_1_0_0\"],null,null,1,1,\"HK:zh-Hant\",null,480,null,null,null,null,null,0,5],\"zh-HK\",\"HK\",1,[2,4,8],1,1,null,0,0,null,0],\"{source}\",{ts},\"{sign}\"]",
        None,
        "generic"
    ]]]
    payload = f"f.req={quote(json.dumps(req_data))}"
    headers = {
      'Host': 'news.google.com',
      # 'Content-Length': '712',
      'X-Same-Domain': '1',
      'Accept-Language': 'zh-CN',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:109.0) Gecko/20100101 Firefox/115.0',
      'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8',
      'Accept': '*/*',
      'Origin': 'https://news.google.com',
      # 'X-Client-Data': 'CPz5ygE=',
      'Sec-Fetch-Site': 'same-origin',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Dest': 'empty',
      'Referer': 'https://news.google.com/',
      'Accept-Encoding': 'gzip, deflate, br',
    }
    response = requests.post(url, headers=headers, data=payload)
    return response.text

In [8]:
# request to each link and extract the <p> tags
final_links = []
for link in l_href[1:]:
    absolute_link = urljoin(url, link)
    
    print(absolute_link)

    
    source, sign, ts = get_google_params(absolute_link)
    final_links.append(get_origin_url(source, sign, ts))



https://news.google.com/read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmpMaDRGU25MQkVzbDA3eXhfQ1AydGR1cFk4RVlxazlzR1JvaXBoVmpUT0tNZXNnX2ZPUEJJS3BmdmxJTGhfdS1KS0UyNzFVRnZhY250eFo1dkROZ0NWb0RzN25OY1JyM0k0OGRwVkJuS1VrRkhSc1FnMGVFQzFwNGtLMA?hl=en-US&gl=US&ceid=US%3Aen
200
https://news.google.com/read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmpMaDRGU25MQkVzbDA3eXh

In [ ]:
final_links

Vamos a probar con selenium

In [22]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [28]:
url = "https://news.google.com/read/CBMivwFBVV95cUxNM2lqN1VsUnluN0dQU0VCZFIxMEowdlNFb3NJSjVNaC1iNUJMNEVxa2xWek5ISFdMY0NiSENRSHNGSHZPc1pRZ29udVI5eEloaDAxZFYyclV3NjFSZ2lnNWZLNTNVZmZJTThnN0IydEhOTmE5dk1XOEFESU5nektRbXllY1F6bU1JdzhFTHhYSkN5MlRXQU5IV3RvYUcwWkk5aGV4ZWVrQW5JTmNTcVR3bEw2d0trNzNqOFRwUi1aSQ?hl=en-US&gl=US&ceid=US%3Aen"

driver = webdriver.Edge()

driver.get(url)

try:
    # Espera a que el botón de 'Accept all' esté presente y clicable
    wait = WebDriverWait(driver, 10)  # Espera hasta 10 segundos
    accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Accept all']")))
    
    # Hace clic en el elemento <span>
    accept_button.click()

    # Esperar a que la nueva página comience a cargarse o hacer algo en el medio si es necesario
    time.sleep(2)  # Pausa opcional para dar tiempo a la carga inicial de la página

    # Scroll hacia abajo hasta el final de la página
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Desplazarse hacia abajo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Esperar a que se cargue la nueva sección
        time.sleep(2)  # Ajusta el tiempo según sea necesario para la página
        
        # Calcular la nueva altura de la página después del scroll
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Si la altura no cambia, hemos llegado al final de la página
        if new_height == last_height:
            break
        
        last_height = new_height
    
    # Una vez que se ha llegado al final, obtener todos los párrafos <p>
    paragraphs = driver.find_elements(By.TAG_NAME, 'p')
    
    # Imprimir el texto de cada párrafo
    for paragraph in paragraphs:
        print(paragraph.text)


        
except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit()



OneFootball's home page
Search
Settings

GiveMeSport
·
6 September 2022
Barcelona’s youth academy, known as La Masia, has produced countless world-class players in the past.
Lionel Messi, Carles Puyol, Xavi and Andres Iniesta are just a few of the legendary names that have come through Barcelona’s academy.
While Gavi and Ansu Fati are two of the names that have graduated from La Masia and are now starring for Barca’s first-team.
The next player that could make the step-up from La Masia to Barca’s first-team is Lamine Yamal.
The youngster, who operates as a winger, is very highly-rated by the Catalan giants.
At just 15 years old, Yamal currently plays for Barca’s Juvenil A team, their U19 side.
Yamal, who is of Moroccan and Equatorial Guinean descent but has played four times for Spain’s U16s, scoring once, has been compared with Lionel Messi at La Masia, per Marca.
He was even dubbed “a mini Leo Messi” by LaLiga’s official Twitter account after they posted a video of his performances 